### Prepare the environment

In [15]:
import pandas as pd
from gensim.models.phrases import Phrases
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_numeric
import re
from gensim import utils
from gensim.corpora.dictionary import Dictionary
from gensim.models import ldamodel
import pyLDAvis.gensim_models as gensim_models
import pyLDAvis
import pyLDAvis.gensim_models

In [16]:
# removed "_" from regular expression
punctuation = r"""!"#$%&'‘()*+,-.\/:;<=>?@[\]^`{|}~"""
RE_PUNCT = re.compile(r'([%s])+' % re.escape(punctuation), re.UNICODE)


def strip_punctuation(s):
    s = utils.to_unicode(s)
    return RE_PUNCT.sub(" ", s)

def prep_clean_data(input_list):
    input_list = [re.sub(r"[^a-zA-Z0-9]", " ", str(i_list)) for i_list in input_list]

    print(input_list[0:10])
    
    #Preprocess
    my_filter = [
        lambda x: x.lower(), strip_tags, strip_punctuation,
        strip_multiple_whitespaces, strip_numeric,
        remove_stopwords, strip_short]

    #bigram = Phrases(persuasion_stream, min_count=1, threshold=2)
    #persuasion_bigrams = ["".join(bigram[i]) for i in persuasion_stream]
    preprocess_clean = [preprocess_string(i, filters=my_filter) for i in input_list]

    print(preprocess_clean[0:5])
    
    return preprocess_clean

### Import dataset

In [17]:
df_text = pd.read_excel(open('data/diffusion_innovation_open_ended.xls','rb'), sheet_name='diffusion_innovation_open_ended')

### Knowledge topic model

In [18]:
knowledge_stream = df_text['knowledge'].tolist()
knowledge_clean = prep_clean_data(knowledge_stream)

['Share insights', 'Have a space that is central in which all person can access  where useful topics and trainings can be accessed ', 'Share examples of how it works', 'Continuous exposure  upskilling and training', 'I would spread the responsibility and put in a great amount of effort in upskilling and obtaining buy in of non IT departments', 'by sending people to training and helping them obtain new skills and making sure they work with the tools as frequent', 'Training individuals and having in house opportunities for them to be involved ', 'Drive business process ownership in a collective fashion  business and IT ', 'implement such systems as part of daily use to make users more familiar with thinking outside the box   e g  in HR systems allow analysis of leave   hours worked or something like that   compare my leave with the average  etc ', '  By improving data quality    Access contro  ']
[['share', 'insights'], ['space', 'central', 'person', 'access', 'useful', 'topics', 'traini

In [19]:
dct_knowledge = Dictionary(knowledge_clean)
corpus = [dct_knowledge.doc2bow(doc) for doc in knowledge_clean]
print(len(dct_knowledge))
print(len(corpus))

434
142


In [20]:
lda = ldamodel.LdaModel(corpus, id2word=dct_knowledge, num_topics = 5)
lda.show_topics(num_words=10)

[(0,
  '0.064*"training" + 0.018*"sharing" + 0.018*"knowledge" + 0.018*"people" + 0.015*"share" + 0.015*"trainings" + 0.015*"workshops" + 0.015*"data" + 0.011*"platforms" + 0.011*"technology"'),
 (1,
  '0.071*"training" + 0.025*"sessions" + 0.021*"online" + 0.021*"product" + 0.020*"sharing" + 0.013*"data" + 0.013*"driven" + 0.013*"awareness" + 0.013*"communication" + 0.012*"knowledge"'),
 (2,
  '0.021*"knowledge" + 0.015*"exposure" + 0.013*"training" + 0.010*"create" + 0.010*"data" + 0.010*"people" + 0.010*"individuals" + 0.010*"technology" + 0.010*"discussion" + 0.010*"contribute"'),
 (3,
  '0.057*"training" + 0.042*"data" + 0.020*"work" + 0.016*"use" + 0.016*"skills" + 0.013*"knowledge" + 0.013*"employees" + 0.013*"technologies" + 0.012*"organization" + 0.010*"people"'),
 (4,
  '0.028*"knowledge" + 0.021*"people" + 0.020*"training" + 0.019*"business" + 0.015*"employees" + 0.015*"skills" + 0.015*"tools" + 0.010*"data" + 0.010*"sharing" + 0.010*"work"')]

In [21]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dct_knowledge)
vis

/home/d5mit/anaconda3/envs/data-driven-org/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/home/d5mit/anaconda3/envs/data-driven-org/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/d5mit/anaconda3/envs/data-driven-org/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/d5mit/anaconda3/envs/data-driven-org/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  f

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.120560  0.024374       1        1  26.543834
0     -0.046458 -0.020743       2        1  22.471601
1     -0.025497  0.102677       3        1  19.245003
2     -0.074744 -0.014814       4        1  15.919033
4      0.026139 -0.091494       5        1  15.820529, topic_info=            Term       Freq      Total Category  logprob  loglift
14      training  41.000000  41.000000  Default  30.0000  30.0000
40      business   3.000000   3.000000  Default  29.0000  29.0000
185      product   4.000000   4.000000  Default  28.0000  28.0000
114       online   5.000000   5.000000  Default  27.0000  27.0000
1          share   3.000000   3.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
135         team   0.738416   1.986453   Topic5  -5.2046   0.8543
105  conferences   0.738306   1.895210   Topic5  -5.2048   0.9011
126   confluence   0.738226   1.911239   Topic5  -5.2049   0.8926
158         need   0.738153   3.324385   Topic5  -5.2050   0.3390
141   management   0.738090   2.665150   Topic5  -5.2051   0.5599

[296 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
332       1  0.725529       able
332       2  0.362765       able
2         4  0.398255     access
2         5  0.398255     access
3         4  0.781485   accessed
...     ...       ...        ...
34        5  0.132901       work
62        1  0.729297     worked
301       2  0.651263  workshops
301       4  0.217088  workshops
301       5  0.217088  workshops

[323 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3, 5])

### Persuasion topic model

In [26]:
persuasion_stream = df_text['persuasion'].tolist()
persuasion_clean = prep_clean_data(persuasion_stream)

['Lead by example  do not lead on opinion', 'Describe the benefits and how it can lead to more informed decisions and the accuracy using a data driven approach rather than gut feeling or based on misinterpreted analysis ', 'Share examples of how it works', 'Introduction presentations and demos of data driven technologies', 'Selling the value and efficiency that it brings  This might require somebody working closely with the dept to understand were the value add lies ', 'training  workshops', 'Awards Certificates for participation and different levels of competency  Possible implementation of ideas brought about through the contribution of the participants ', 'Highlight competitive advantage  highlight the existence of trends hidden in data  highlight the criticality of decisions based on trusted information  highlight the need to have an appetite for change  willingness to change direction based on observations within the data  ', 'have a prominent display section in for instance the d

In [27]:
dct_knowledge = Dictionary(persuasion_clean)
corpus = [dct_knowledge.doc2bow(doc) for doc in persuasion_clean]
print(len(dct_knowledge))
print(len(corpus))

518
142


In [28]:

lda = ldamodel.LdaModel(corpus, id2word=dct_knowledge, num_topics = 4)
lda.show_topics(num_words=10)
    

[(0,
  '0.025*"benefits" + 0.023*"data" + 0.013*"teams" + 0.013*"incentives" + 0.010*"showing" + 0.010*"real" + 0.009*"need" + 0.009*"learning" + 0.008*"awareness" + 0.007*"driven"'),
 (1,
  '0.032*"data" + 0.029*"value" + 0.028*"people" + 0.011*"driven" + 0.009*"create" + 0.009*"demonstrate" + 0.009*"understand" + 0.009*"work" + 0.009*"proof" + 0.008*"workshops"'),
 (2,
  '0.054*"data" + 0.028*"driven" + 0.014*"benefits" + 0.013*"training" + 0.010*"highlight" + 0.010*"examples" + 0.009*"use" + 0.009*"based" + 0.009*"results" + 0.008*"awareness"'),
 (3,
  '0.039*"data" + 0.019*"driven" + 0.013*"use" + 0.012*"value" + 0.010*"user" + 0.008*"management" + 0.008*"decisions" + 0.008*"results" + 0.007*"result" + 0.007*"tech"')]

In [29]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dct_knowledge)
vis


/home/d5mit/anaconda3/envs/data-driven-org/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.070288 -0.074779       1        1  32.355019
0     -0.102906 -0.029676       2        1  22.920440
3      0.009498  0.015619       3        1  22.531410
1      0.023120  0.088836       4        1  22.193131, topic_info=             Term       Freq      Total Category  logprob  loglift
126        people   7.000000   7.000000  Default  30.0000  30.0000
32          value  10.000000  10.000000  Default  29.0000  29.0000
189    incentives   3.000000   3.000000  Default  28.0000  28.0000
7        benefits  11.000000  11.000000  Default  27.0000  27.0000
114         teams   4.000000   4.000000  Default  26.0000  26.0000
..            ...        ...        ...      ...      ...      ...
438   performance   0.902703   1.860413   Topic4  -5.4407   0.7822
82        success   1.235950   3.728842   Topic4  -5.1265   0.4011
10         driven   2.373222  16.641156   Topic4  -4.4741  -0.4422
21   technologies   1.010457   3.078470   Topic4  -5.3280   0.3914
7        benefits   1.328766  11.971032   Topic4  -5.0541  -0.6929

[242 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
358       3  0.538856       able
195       3  0.530376   accurate
288       3  0.536146   achieved
346       3  0.530412   actually
22        3  0.421264        add
...     ...       ...        ...
106       2  0.275804       work
106       4  0.551609       work
35        1  0.454998  workshops
35        3  0.227499  workshops
35        4  0.454998  workshops

[275 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])